# 작사가 인공지능 만들기

## 데이터 다듬기

In [1]:
import os, re
import numpy as np
import tensorflow as tf

import glob
import os

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]


In [2]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue 
    if sentence[-1] == ":": continue 

    if idx > 9: break  
        
    print(sentence)

Now I've heard there was a secret chord
That David played, and it pleased the Lord
But you don't really care for music, do you?
It goes like this
The fourth, the fifth
The minor fall, the major lift
The baffled king composing Hallelujah Hallelujah
Hallelujah
Hallelujah
Hallelujah Your faith was strong but you needed proof


In [3]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


In [4]:
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
corpus[:10]

['<start> now i ve heard there was a secret chord <end>',
 '<start> that david played , and it pleased the lord <end>',
 '<start> but you don t really care for music , do you ? <end>',
 '<start> it goes like this <end>',
 '<start> the fourth , the fifth <end>',
 '<start> the minor fall , the major lift <end>',
 '<start> the baffled king composing hallelujah hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah your faith was strong but you needed proof <end>']

In [5]:
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words= 12000,
        filters=' ',
        oov_token="<unk>"
    )

    tokenizer.fit_on_texts(corpus)
    tensor = tokenizer.texts_to_sequences(corpus)   
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen = 15)  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2   50    5 ...    0    0    0]
 [   2   17 2639 ...    0    0    0]
 [   2   36    7 ...   43    3    0]
 ...
 [   5   22    9 ...   10 1013    3]
 [  37   15 9049 ...  877  647    3]
 [   2    7   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7faf1c4ac100>


In [6]:
print(tensor[:3, :10])

[[   2   50    5   91  297   65   57    9  969 6042]
 [   2   17 2639  873    4    8   11 6043    6  329]
 [   2   36    7   37   15  164  282   28  299    4]]


In [7]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


In [8]:
src_input = tensor[:, :-1]
tgt_input = tensor[:, 1:]

print(src_input[0])
print(tgt_input[0])

[   2   50    5   91  297   65   57    9  969 6042    3    0    0    0]
[  50    5   91  297   65   57    9  969 6042    3    0    0    0    0]


In [9]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1   

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

In [10]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
lyricist = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [11]:
from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2, random_state=77)

## 인공지능 학습시키기

In [13]:
for src_sample, tgt_sample in dataset.take(1): break

lyricist(src_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[-1.55151894e-04, -7.37548471e-05, -3.34326382e-04, ...,
          3.40311410e-04,  8.85685949e-05,  2.41720103e-04],
        [ 5.67520765e-05, -8.40629655e-05, -5.51901117e-04, ...,
          4.26964776e-04, -7.20167227e-05,  4.48473613e-04],
        [ 2.46885786e-04, -2.49523378e-04, -5.06411132e-04, ...,
          4.43717552e-04, -3.75943579e-04,  6.63637766e-04],
        ...,
        [-6.91201421e-04,  1.00111403e-03,  1.02094002e-03, ...,
          1.83974043e-03,  1.16257183e-03,  1.05904233e-04],
        [-6.33073447e-04,  9.23938351e-04,  1.24200224e-03, ...,
          1.76542182e-03,  1.13758212e-03, -1.23777412e-04],
        [-3.96912685e-04,  6.79556921e-04,  1.36858283e-03, ...,
          1.61755783e-03,  1.00647600e-03, -3.06890463e-04]],

       [[-1.55151894e-04, -7.37548471e-05, -3.34326382e-04, ...,
          3.40311410e-04,  8.85685949e-05,  2.41720103e-04],
        [-1.79549970e-04, -1.09416578e-04, -3

In [14]:
lyricist.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  3072256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  12301025  
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


In [16]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

lyricist.compile(loss=loss, optimizer=optimizer)
lyricist.fit(dataset, epochs=10)

Epoch 1/10
686/686 [==============================] - 115s 165ms/step - loss: 3.6114
Epoch 2/10
686/686 [==============================] - 113s 164ms/step - loss: 3.1217
Epoch 3/10
686/686 [==============================] - 113s 164ms/step - loss: 2.9337
Epoch 4/10
686/686 [==============================] - 113s 164ms/step - loss: 2.7889
Epoch 5/10
686/686 [==============================] - 113s 164ms/step - loss: 2.6617
Epoch 6/10
686/686 [==============================] - 113s 164ms/step - loss: 2.5459
Epoch 7/10
686/686 [==============================] - 113s 165ms/step - loss: 2.4381
Epoch 8/10
686/686 [==============================] - 113s 165ms/step - loss: 2.3368
Epoch 9/10
686/686 [==============================] - 113s 165ms/step - loss: 2.2404
Epoch 10/10
686/686 [==============================] - 113s 165ms/step - loss: 2.1488


## 잘 만들어졌는지 평가하기

In [20]:
def generate_text(lyricist, tokenizer, init_sentence="<start>", max_len=20):
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    while True:
    
        predict = lyricist(test_tensor)
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [26]:
generate_text(lyricist, tokenizer, init_sentence="<start> i love", max_len=15)

'<start> i love you so much <end> '

# 회고


 알파고가 한창 이슈였던 때인 2016년 쯤을 돌이켜 보면, 저는 인공지능 즉 AI의 등장으로 없어질 직업이라는 기사가 떠오릅니다. 제 기억으로는 순위권에 경리와 회계사 등이 있었는데, 이번 Exploration의 주제인 작사가도 그 중 하나였습니다.
 
 AI가 번역을 해낸다는 것은 '뭐 그럴 수 있지?'라 생각하지만, 그 당시 AI가 글을 쓴다는 것은 상상하기 어려웠습니다. 물론 챗봇을 기획해보며, 무엇보다도 AIFFEL의 일원으로서 AI가 작사를 할 수 있다는 것을 너무나 당연히 알고 있습니다. 제가 어떤 과정을 거쳤는지에 대해서 그리고 어떤 부분을 새롭게 배웠는지 회고해보고자 합니다.
 
 제가 작곡한 글자는 "i love you so much"입니다. 어떻게 보면 그럴 듯합니다. 누구나 작사에 사용하는 문구겠지만, 누구나 쓰기에 AI가 인간의 공감을 얻을 수 있는 작사가이지 않나라고 생각합니다.
 
 먼저, 이번 Exploration에서 개인적으로 중요하다고 생각한 부분은 데이터 전처리 과정에서 부호 및 특수문자와 대문자 및 소문자를 정규표현식으로 필터링하는 것이라 생각합니다. 왜냐하면, 가장 기본이니깐? 처음 알게 된 방법이라서 그렇습니다. 서두에 말씀드린 것처럼 작사는 안될 거라는 것을 된다고 알려줬기 때문입니다. 이때 특별히 어려움은 없었기에 다음으로 진행했습니다.
 
 다음으로, 텍스트 생성모델이 안정적으로 학습되게 하기 위해서 'num_words= 12000'를 조절해 validation loss를 2.2이하로 맞췄습니다. 또한, 문장 길이를 15를 최대로 맞추려고 'tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen = 15)'를 추가해주었습니다.
 
 going deeper에서 NL를 고민하고 있는 저에게 좋은 프로젝트였다고 생각합니다. 이를 계기로 조금 더 나은 선택을 할 수 있고, 아이디어를 떠올릴 수 있을 것 같습니다. 